In [1]:
import math, copy, time
import torch
from TD3 import TD3

from utils import ReplayBuffer, episode_stats_queue
import numpy as np
import mujoco as mj
import mujoco.viewer as vi
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Mujoco Environment

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tues Nov 14 1:20:12 2023

@author: Kartik Nagpal, Jean-Baptiste Bouvier

Simple environment for the mujoco system, inspired from the gym environment.
"""

class MujocoEnv():
    """Mujoco system environment."""
    
    def __init__(self, target = torch.tensor([0.5, 0.5, 0.5], device=device),
                       precision = 0.05,
                       max_magnitude = 0.1,
                       constraint_min = torch.tensor([0.41, 0.34, 0.57], device=device),
                       constraint_max = torch.tensor([0.59, 0.66, 0.57], device=device),
                       debug = False):

        ### Environment Parameters
        self.state_size = 10 # state dimension
        self.action_size = 7 # input dimension
        self.max_actuation = torch.tensor([2.97, 2.09, 2.97, 2.09, 2.97, 2.09, 3.05], device=device)
        self.action_min = -1*self.max_actuation
        self.action_max = 1*self.max_actuation
        self.last_action = torch.zeros(1, self.action_size)
        self.action_magnitude = max_magnitude

        ### Mujoco Setup
        self.model = mj.MjModel.from_xml_path("./robotic-arm-model/test_scene.xml")
        self.data = mj.MjData(self.model)
        self.viewer = vi.launch_passive(self.model, self.data)
        cam = self.viewer.cam
        cam.azimuth = 145.0 ; cam.elevation = -19.25 ; cam.distance = 2.0
        cam.lookat = np.array([ 0.2228265373149372 , 0.25857197832292195 , 0.7914213541765485 ])
        
        ### Target
        self.target = target
        self.precision = precision

        ### Forbidden to cross the line between constraint_min and constraint_max
        self.constraint_min = constraint_min
        self.constraint_max = constraint_max
        self.width_affine_area = 2*torch.tensor([0.09, 0.16, 0.05], device=device)

        self.constraint_line_x1 = torch.tensor([constraint_min[0], constraint_min[1]], device=device)
        self.constraint_line_x3 = torch.tensor([constraint_min[0], constraint_max[0]], device=device)
        self.constraint_line_x2 = torch.tensor([constraint_min[1], constraint_max[1]], device=device)
        self.constraint_line_x4 = torch.tensor([constraint_max[0], constraint_max[1]], device=device)
        self.constraint_max[2] += self.width_affine_area[2] # Switching the corner
        self._allowed_constraint_area()
        
        self.site_index = -2
        
        self.debug = debug
        

    def _allowed_constraint_area(self):
        """Calculates where it is allowed."""
        self.allowed_constraint_area = torch.cat((torch.cat(self.constraint_min, self.action_min),
                                                  torch.cat(self.constraint_max, self.action_max),
                                                  self.constraint_max + self.width_affine_area,
                                                  self.constraint_min + torch.tensor([self.width_affine_area[0:2], 0.0])))

    def getJoints(self):
        return torch.tensor(self.data.qpos, dtype=torch.float, device=device)
    
    def reward(self, state):
        r = -torch.sum(torch.abs(state - self.target))
        return r

    def fullState(self):
        # Based on the current state definition
        return copy.deepcopy(torch.cat((self.state, self.last_action)))


    
    def crossing(self, new_state):
        if((new_state > self.constraint_min).all() and (new_state < self.constraint_max).all()):
            return True
        if(torch.sign(new_state[2]-self.constraint_min[2]) != torch.sign(self.state[2]-self.constraint_min[2])):
            if(new_state[0] > self.constraint_min[0] and new_state[0] < self.constraint_max[0]):
                if(new_state[1] > self.constraint_min[1] and new_state[1] < self.constraint_max[1]):
                    return True
        return False

    def respect_constraint(self, new_state):
        """Checks whether new_state respects the constraint."""
        return not self.crossing(new_state)
    
    
    def arrived(self):
        """Episode is over when target is reached within given precision."""
        return torch.linalg.vector_norm(self.target - self.state) < self.precision
    
    def _propagation(self, u):
        """Common background function for 'step' and 'training_step'."""
        u = torch.clamp(u, min=-self.action_magnitude, max=self.action_magnitude)
        
        self.data.qpos += u.cpu().numpy()
        self.last_action = torch.tensor(self.data.qpos, dtype=torch.float, device=device)
        mj.mj_step(self.model, self.data)
        new_state = torch.tensor(self.data.site_xpos[self.site_index], dtype=torch.float, device=device)
        respect = self.respect_constraint(new_state)
        too_far = (self.last_action < self.action_min).any().item() or (self.last_action > self.action_max).any().item() or (new_state[2] < 0)

        if(self.debug):
            self.viewer.sync()
            print("Propogation function:", u, new_state, respect, too_far)

        return new_state, respect, too_far
            
    def step(self, u):
        """Returns: new_state, reward, done, respect"""
        self.last_state = self.state
        new_state, respect, too_far = self._propagation(u)
        
        ### Update to new state only if constraint is not violated and hasn't exited allowed area
        if not too_far and respect:
            self.state = new_state
        else:
            self._propagation(-u)
            # print("Violation!")
        
        done = self.arrived()
        reward = self.reward(self.state) + done - 2*too_far - 6*(not respect)
        reward -= 20*torch.relu(self.last_action.abs().max(dim=1).values).item()

        if(self.debug):
            self.viewer.sync()
            print("Step function: ", u, self.state, respect, too_far, reward)

        return self.fullState(), reward, done, respect

    def training_step(self, u):
        """Takes a step in the environment. If the constraint is violated or if
        the states exits the admissible area, the state is maintained in place
        and a penalty is assigned. This prevents too short trajectories.
        'done' is only assigned if the target is reached."""
        return self.step(u)
        

    def reset(self, max_dist_target = None): # base value should allow full state space sampling
        """Resets the state of the linear system to a random state in [x_min, x_max]."""
        self.last_action = (self.action_max - self.action_min) * torch.rand(self.action_size, device=device) + self.action_min

        self.data.qpos = self.last_action.cpu()
        mj.mj_step(self.model, self.data)
        self.viewer.sync()
        self.state = torch.tensor(self.data.site_xpos[self.site_index], dtype=torch.float, device=device)
        self.last_state = self.state
                
        if max_dist_target is not None and torch.norm(self.state - self.target) > max_dist_target:
            self.reset(max_dist_target)
            return self.fullState()

        if(self.debug):
            self.viewer.sync()
            print(self.last_action, self.state, torch.norm(self.state - self.target))

        return self.fullState()
    
    def random_action(self):
        """Returns a uniformly random action within action space."""
        a = (self.action_max - self.action_min) * torch.rand(self.action_size) + self.action_min
        return torch.clamp(a, min=-self.action_magnitude, max=self.action_magnitude, device=device)

    def close(self):
        self.viewer.close()
                

Policy Definitions

In [3]:
#%% Hyperparameters
POLICEd = False
N_step = 100                     # Length of each episode
rand_episodes = 0                # Number of episodes where random policy is used
num_episodes = 2500              # Total number of training episodes


policy_freq = 3                  # Frequency in episodes of delayed policy updates
in_constraint_freq = 100       # Frequency in episodes at which the initial state is chosen in the allowed constraint area

hidden_size = 128                # Number of units per hidden layer
batch_size = 512                 # Batch size for both actor and critic
discount = 1.00                  # Discount factor
tau = 0.01                       # Target network update rate

prb_alpha = 1                    # Prioritized Experience Replay alpha value
reward_threshold = 1000          # Minimum average reward to stop training
buffer_priority_method = ""

### Cornered target
target = torch.tensor([[0.5, 0.5, 0.5]], device=device)
precision = 0.1
max_action_magnitude = 0.05
debug = False

# Mujoco Environment that we defined
env = MujocoEnv(target=target, precision=precision, max_magnitude=max_action_magnitude, debug=debug)

# Noise values for TD3
noise = torch.full((env.action_size,), env.action_magnitude, device=device)
expl_noise = 0.025*noise        # Std of Gaussian exploration noise
policy_noise = 0.025*noise      # Noise added to target policy during critic update
noise_clip = 0.05*noise        # Range to clip target policy noise

noise = torch.full((env.action_size,), 0.0, device=device)
expl_noise = 0.025*noise        # Std of Gaussian exploration noise
policy_noise = 0.025*noise      # Noise added to target policy during critic update
noise_clip = 0.05*noise        # Range to clip target policy noise


hyperparameters = {
    "buffer_priority_method": buffer_priority_method,
    "POLICEd": POLICEd,
    "N_step": N_step,
    "rand_episodes": rand_episodes,
    "num_episodes": num_episodes,
    "prb_alpha": prb_alpha,
    "policy_freq": policy_freq,
    "hidden_size": hidden_size,
    "batch_size": batch_size,
    "discount": discount,
    "tau": tau,
    "reward_threshold": reward_threshold,
    "target": target
}

print(hyperparameters)


controller = TD3(env, POLICEd, hidden_size, discount, tau, policy_noise, noise_clip, policy_freq)
replay_buffer = ReplayBuffer(env.state_size, env.action_size, max_size=1000*N_step, priority_method=buffer_priority_method, alpha=prb_alpha)

{'buffer_priority_method': '', 'POLICEd': False, 'N_step': 100, 'rand_episodes': 0, 'num_episodes': 2500, 'prb_alpha': 1, 'policy_freq': 3, 'hidden_size': 128, 'batch_size': 512, 'discount': 1.0, 'tau': 0.01, 'reward_threshold': 1000, 'target': tensor([[0.5000, 0.5000, 0.5000]], device='cuda:0')}
device:  cuda


### Training

In [ ]:
stats_queue = episode_stats_queue(max_size=100)
trained = False
min_reward = -100000
PATH = "./checkpoints/rss_pos_3.pt"
all_stats = np.zeros((num_episodes, 3))

for episode in range(num_episodes):
    # state = env.reset(episode/num_episodes + 0.3)
    state = env.reset()
    episode_reward = 0
    episode_constraint_respect = True
    episode_completion = False

    for t in range(N_step):
        ### Select action randomly or according to policy
        if episode < rand_episodes:
            action = env.random_action()
        else:
            with torch.no_grad():
                action = controller.select_action(state)
                action += torch.randn_like(action) * expl_noise

        ### Perform action
        next_state, reward, done, respect = env.training_step(action)
        ### Store data in replay buffer
        replay_buffer.add(state, action, next_state, reward, float(done))
        state = next_state
        
        env.viewer.sync()
        
        episode_reward += reward
        if not respect:
            episode_constraint_respect = False

        ### Train agent after collecting sufficient data
        if episode >= rand_episodes:
            controller.train(replay_buffer, batch_size)
        if done: 
            episode_completion = True
            break

    all_stats[episode] = [episode_reward.cpu(), episode_constraint_respect, episode_completion]
    stats_queue.add(episode_reward, float(episode_constraint_respect), float(episode_completion))
    average_reward, average_respect = stats_queue.average()
    ### Evaluate episodes
    if episode % 50 == 0:
        print(f"Episode: {episode} \t Average reward: {average_reward:.2f} \t Average respect: {average_respect:.2f} \t Percent Completion: {stats_queue.percentCompletion():.2f} %")
        # violation = empirical_constraint_violation(env, controller, 1000, True)
        trained = average_reward > reward_threshold # and violation == 0.
        # policy_map(env, controller, f"Episode {episode}:")
        if(trained):
            break
    elif episode % 10 == 0:
        # violation = empirical_constraint_violation(env, controller, 100, False)
        trained = average_reward > reward_threshold # and violation == 0.
        if(average_reward > min_reward):
            torch.save({
                'epoch': episode,
                'actor_state_dict': controller.actor.net.state_dict(),
                'actor_optimizer': controller.actor_optimizer.state_dict(),
                'critic_Q1_state_dict': controller.critic.Q1_net.state_dict(),
                'critic_Q2_state_dict': controller.critic.Q2_net.state_dict(),
                'critic_optimizer': controller.critic_optimizer.state_dict(),
                'average_reward': average_reward,
                'average_respect': average_respect,
                'all_rewards': all_stats,
                'metadata': hyperparameters
                }, PATH)
            min_reward = average_reward
        # elif(average_reward < min_reward-100): # Stop training if it's not working
        #     trained = True
        if(trained):
            print(f"FINISHED TRAINING - Episode: {episode} \t Average reward: {average_reward:.2f} \t Average respect: {average_respect:.2f} %")
            break

### Trajectory Rollout

In [ ]:
# Rolling out a trajectory
state = env.reset()
env.viewer.sync()
episode_reward = 0

N_step=100

for t in range(N_step):
    # step_start = time.time()
    with torch.no_grad():
        action = controller.select_action(state)
    print(state.cpu())#, action.cpu())
    state, reward, done, respect = env.step(action)
    # print(reward, env.reward(state[:3])) #reward,
    reward = -torch.sum(torch.abs(state[:3] - env.target))
    episode_reward += reward

    time.sleep(0.1)
    env.viewer.sync()
    
    if done:
       print("DONE")
       break



#### Manual Save Checkpoint

In [ ]:
## Saving and Loading checkpoints
average_reward, average_respect = stats_queue.average()
PATH = "./model.pt"

torch.save({
    'epoch': num_episodes,
    'actor_state_dict': controller.actor.net.state_dict(),
    'actor_optimizer': controller.actor_optimizer.state_dict(),
    'critic_Q1_state_dict': controller.critic.Q1_net.state_dict(),
    'critic_Q2_state_dict': controller.critic.Q2_net.state_dict(),
    'critic_optimizer': controller.critic_optimizer.state_dict(),
    'average_reward': average_reward,
    'average_respect': average_respect,
    }, PATH)

In [6]:
env.close()

#### Load Checkpoint

In [7]:
PATH = "./checkpoints/unconstrained_full_6_uncliped.pt"

env = MujocoEnv(target=target, precision=precision, max_magnitude=max_action_magnitude, debug=debug)
controller = TD3(env, POLICEd, hidden_size, discount, tau, policy_noise, noise_clip, policy_freq)
replay_buffer = ReplayBuffer(env.state_size, env.action_size, max_size=1000*N_step, priority_method=buffer_priority_method, alpha=prb_alpha)

checkpoint = torch.load(PATH)
controller.actor.net.load_state_dict(checkpoint['actor_state_dict'])
controller.actor_optimizer.load_state_dict(checkpoint['actor_optimizer'])
controller.critic.Q1_net.load_state_dict(checkpoint['critic_Q1_state_dict'])
controller.critic.Q2_net.load_state_dict(checkpoint['critic_Q2_state_dict'])
controller.critic_optimizer.load_state_dict(checkpoint['critic_optimizer'])
epoch = checkpoint['epoch']
average_reward = checkpoint['average_reward']
average_respect = checkpoint['average_respect']
print(average_reward.cpu(), average_respect)

device:  cuda
tensor(-67.1876) 0.8169014084507042


In [ ]:
data = checkpoint['all_rewards']
data = data[data[:,0] != 0., :]
data = np.insert(data, 0, 100, axis=1)
print(data)

np.savetxt("./plotting/POLICEd_3.csv", data, delimiter=",")
env.close()

#### Evaluation Suite

In [40]:
import scipy.stats

num_episodes = 500
eval_stats = np.zeros((num_episodes, 3))


for episode in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    episode_constraint_respect = True
    episode_completion = False

    for t in range(N_step):
        ### Select action randomly or according to policy
        with torch.no_grad():
            action = controller.select_action(state)
            # action += torch.randn_like(action) * expl_noise

        ### Perform action
        next_state, reward, done, respect = env.training_step(action)
        ### Store data in replay buffer
        state = next_state
        episode_reward += reward
        
        if not respect:
            episode_constraint_respect = False
        if done:
            episode_completion = True
            break

    eval_stats[episode] = [episode_reward.cpu(), episode_constraint_respect, episode_completion]

def get_confidence_interval(data, confidence=0.95):
    m, se = np.mean(data), scipy.stats.sem(data)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., len(data)-1)
    return m, m-h, m+h, h


print("Reward CI:" + str(get_confidence_interval(eval_stats[:,0])))
print("Respect CI:" + str(get_confidence_interval(eval_stats[:,1])))
print("Percent Completion:" + str(100*sum(eval_stats[:,2])/num_episodes))

#print(f"Reward CI: {get_confidence_interval(eval_stats[:,0])} \t Average respect: {get_confidence_interval(eval_stats[:,1])} \t Percent Completion: {100*sum(eval_stats[:,2])/num_episodes} %")


Reward CI:(-25.02001190185547, -25.02001190185547, -25.02001190185547, 0.0)
Respect CI:(1.0, 1.0, 1.0, 0.0)
Percent Completion:0.0
